In [23]:
from keras.constraints import Constraint
import keras.backend as K


#this function just clips values to a range (elementwise)
class Between(Constraint):
    def __init__(self, min_value, max_value):
        self.min_value = min_value
        self.max_value = max_value

    def __call__(self, w):        
        return K.clip(w, self.min_value, self.max_value)

    def get_config(self):
        return {'min_value': self.min_value,
                'max_value': self.max_value}

In [24]:
import keras
import numpy as np
from keras import backend as K

from keras.layers import Layer
from keras.layers import activations
from keras.layers import initializers
from keras.layers import regularizers
from keras.layers import constraints


from keras.utils import to_categorical
from  keras.engine.base_layer import InputSpec
from keras.utils.generic_utils import to_list

class Tent(Layer):
    """Tent activation Unit.
    It follows:
    `f(x) =  max( 0, theta -|x|) ,

    where `theta` is a learned array with the same shape as x.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    # Output shape
        Same shape as the input.
    # Arguments
        theta_initializer: initializer function for the weights.
        theta_regularizer: L2 regularization strenth
        theta_max: highest allowed value for theta (min value is set to 0.05)
        shared_axes: the axes along which to share learnable
            parameters for the activation function.
            For example, if the incoming feature maps
            are from a 2D convolution
            with output shape `(batch, height, width, channels)`,
            and you wish to share parameters across space
            so that each filter only has one set of parameters,
            set `shared_axes=[1, 2]`.
    # References
    https://arxiv.org/pdf/1908.02435.pdf
    """

    def __init__(self,
                 theta_regularizer=0.12,
                 theta_max=1.0,
                 shared_axes=None,
                 **kwargs):
        super(Tent, self).__init__(**kwargs)
        self.supports_masking = True # Do not know what this does, just let it be
        self.theta_initializer = initializers.Ones() #see article
        self.theta_regularizer = regularizers.l2(theta_regularizer) # I interpreted "weight decay" as l2, not l1 
        self.theta_constraint = Between(min_value=0.05,max_value=theta_max)

        if shared_axes is None:
            self.shared_axes = None
        else:
            self.shared_axes = to_list(shared_axes, allow_tuple=True)

    def build(self, input_shape):
        param_shape = list(input_shape[1:])
        self.param_broadcast = [False] * len(param_shape)
        if self.shared_axes is not None:
            for i in self.shared_axes:
                param_shape[i - 1] = 1
                self.param_broadcast[i - 1] = True
        self.theta = self.add_weight(shape=param_shape,
                                     name='theta',
                                     initializer=self.theta_initializer,
                                     regularizer=self.theta_regularizer,
                                     constraint=self.theta_constraint)
        # Set input spec
        axes = {}
        if self.shared_axes:
            for i in range(1, len(input_shape)):
                if i not in self.shared_axes:
                    axes[i] = input_shape[i]
        self.input_spec = InputSpec(ndim=len(input_shape), axes=axes)
        self.built = True

    def call(self, inputs, mask=None):
        pos = K.relu(self.theta - K.abs(inputs))
        return pos 

    #TODO: what is this config for? Proabably should update this to add "theta_max"
    def get_config(self):
        config = {
            'theta_initializer': initializers.serialize(self.theta_initializer),
            'theta_regularizer': regularizers.serialize(self.theta_regularizer),
            'shared_axes': self.shared_axes
        }
        base_config = super(Tent, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape

# Now lets try to test the froward pass
#TODO: maybe make it a graph? Anyway, who cares, we proved it works

In [25]:
from keras.layers import Input
from keras.models import Model

inp = Input(shape=(10,))
out = Tent()(inp)
model = Model(inp, out)

output = model.predict(np.stack([np.arange(-2.0,2.0,0.4),np.arange(-1.1,1.1,0.22)]))
print(output)

[[0.         0.         0.         0.19999999 0.6        1.
  0.6        0.19999999 0.         0.        ]
 [0.         0.12       0.33999997 0.56       0.78       1.
  0.78       0.56       0.33999997 0.12      ]]


# Let's see if thetas can be shared

In [26]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D


model = Sequential()
model.add(Dense(100)) 
model.add(Tent(shared_axes=[0]))  #returns a list of 1 value only
#model.add(Tent()) #returns a list of 100 vals
model.add(Dense(1))
model.compile(loss='mse',optimizer='adam')
model.fit(np.random.random((100,100)),np.random.random((100,1)),epochs=1)

Epoch 1/1
100/100 [==============================] - 0s 4ms/step - loss: 3.4885


In [27]:
model.layers[1].get_weights()

[array([0.9961924], dtype=float32)]

# try it with conv2D


In [37]:
from keras.utils import to_categorical

model = Sequential()
model.add(Conv2D(3,(2,2),padding="valid", input_shape=(2,6,3))) 
model.add(Tent(shared_axes=[0,1,2]))  #returns a list of 1 value only
#model.add(Tent(shared_axes=[1,2]))  # returns 1 value per filter
#model.add(Tent()) #returns one val per output
model.add(Flatten())
model.add(Dense(10))
model.compile(loss='categorical_crossentropy',optimizer='adam')
model.fit(np.random.random((100,2,6,3)),to_categorical(np.random.choice(range(10),100)),epochs=1)

Epoch 1/1
100/100 [==============================] - 1s 6ms/step - loss: 8.3261


In [35]:
print(model.layers[1].get_weights())
print("the shape of tent  ", np.shape(model.layers[1].get_weights()))
print("the shape of output featuer maps ", np.shape(model.layers[1].output))

[array([[[1.       , 0.9963166, 0.9974665]]], dtype=float32)]
the shape of tent   (1, 1, 1, 3)
the shape of output featuer maps  (?, 1, 5, 3)


# TODO: test adding constraint to TENT works
(no idea how to do it properly. train something and see that boundaries never go beyond?)

# TODO: test regularization works 
(put mad regularization and see it will give widhts 0.05)